In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from urllib.request import urlopen
import folium
import branca
import scipy as sp
import scipy.ndimage
import geojsoncontour
from math import radians
from folium import plugins
from scipy.interpolate import griddata
from sklearn.metrics.pairwise import haversine_distances

import warnings
warnings.filterwarnings("ignore")
!pip install openpyxl

%matplotlib inline

# Social data analysis and visualizations - Final project
## Recycling behaviour in New York City

Here you can find a link to our website: ..

NOTE: the notbooks need to be downloaded and press "Trusted" in the right corner, for all the plots to render correctly.

### Motivation

Recycling is a key component of modern waste reduction and one of the main aspects of green cities. It promotes environmental sustainability by removing raw material input and redirecting waste output in the economic system. Recycling converts the things we throw away into new items, making sure that none of the energy and raw materials used to make them go to waste. It also prevents air and ground pollution, and the release of greenhouse gases that results from dumping waste on landfill sites. But recycling can achieve even more things.
Recycling items rather than using raw materials to make new things preserve the planet’s natural resources which, in the face of population growth and growing demand, won’t last forever. Recycling materials uses less energy than extracting, processing, and transporting raw materials to make new products. Think about how raw materials are usually extracted, and what harm these activities might do to the earth. Mining, quarrying, logging, and fracking all cause harm to the planet by causing air and water pollution. These activities can also destroy precious animal habitats. Even more, recycling reduces the amount of waste we send to landfills. When waste sits rotting away in landfills, it leeches toxins into the groundwater and soil, and gives off greenhouse gases like methane as it decomposes, which contributes to global warming. Not only that, if recyclable items are sent to landfills, the precious raw materials, and energy that went into making them are lost. 
These are only some of the reasons that we find interesting in recycling.



The following video summarizes the main idea behind our project:

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('ul26sZWu4Po')

##### What is the idea?

This project aims to determine how much waste New York City (NYC) residents recycle and whether recycling varies among different districts in NYC. If a difference does exist, we hope to uncover possible explanations for it. Moreover, different socialeconomic factors are also going to be investigated to se if there is any kind of correlation between recycling and these factors.

##### What are your datasets?

For our analysis we choose some different types of dataset to have a more clear view of our problem and to investigate it through different aspects. We will refer to its dataset and the variables later in this notebook.

[Recycling data](https://data.cityofnewyork.us/Environment/Recycling-Diversion-and-Capture-Rates/gaq9-z3hz): Monthly data for 2018 and 2019 by district; 145 kB, 1000 rows, 9 variables.<br>
[District data (district locations)](https://data.cityofnewyork.us/City-Government/DSNY-Districts/i6mn-amj2): Geo-data of districts; 5 mB, 59 rows, 7 variables.<br>
[Public recycling bins data](https://data.cityofnewyork.us/Environment/Public-Recycling-Bins/sxx4-xhzg):  Geo-data of recycling bins in New York; 47 kB, 545 rows, 6 variables.<br>
[Demographic data](https://data.cityofnewyork.us/City-Government/Community-District-Breakdowns/w3c6-35wg): Average data across 2015 - 2019 by district; 508 kB, 59 rows, 485 variables<br>
[Social data](https://www1.nyc.gov/site/planning/planning-level/nyc-population/american-community-survey.page.page): Average data across 2015 - 2019 by district; 1 mB, 59 rows, 2191 variables.<br>
[Economic data](https://www1.nyc.gov/site/planning/planning-level/nyc-population/american-community-survey.page.page): Average data across 2015 - 2019 by district; 325 kB, 59 rows, 660 variables.<br>
[Housing data](https://data.cityofnewyork.us/City-Government/Demographic-Social-Economic-and-Housing-Profiles-b/kvuc-fg9b): Average data across 2015 - 2019 by district; 281 kB, 59 rows, 520 variables.<br>

##### Why is it interesting?

Considering that recycling levels differ among different districts in New York City, it would be very interesting to learn the reasons/explanations behind these differences. Understanding why recycling is successful in some areas and unsuccessful in others may inspire new initiatives to promote recycling in New York City. In addition to being great for the environment, it would also benefit NYC's residents (recycling reduces pollution, conserves energy, reduces the need for new raw materials, and keeps trash out of landfills). If our findings are meaningful then propably the same paterns would be applied to other cities in the US and/or around the world also. This means that at the end we can generalize our findings. 

This project would mostly affect UN SDGs 11 and 12 in a positive way.

##### What was your goal for the end user's experience?

Our main goal is to communicate ouf findings through a public access web-page. With interactive plots we want the users to choose by themselves different socioeconomic factors and see on maps how they affect the recycling behavior of each neighborhood. Plots of data will also be provided to make the user familiar with them as well as other plots that we might find interesting to share when we proceed with our analysis. Last our conclusions will also be presented to complete our story.


### Basic stats. Let's understand the dataset better

#### Write about your choices in data cleaning and preprocessing

Before we start analyzing our data we had in some cases to perform some data preprocessing. The first issue that we had to solve was that some of the datasets had the districts written in a different format from the others. This would be a problem for our analysis since later we wanted to compine and compare our datasets. We made a function in order to change the "wrong" names and have the same ones in all datasets. The second problem we faced with some sets was that they contained too many variables in most of the cases irrelevant with our problem. Choosing the variables we need was part of the data preparation. Finaly, we had to deal with some None values that we either dropped in some cases or we filled the with the mean value of the specific row.  

In [ ]:
def clean_district(district):
    if 'BKN' in district:
        district = district.replace('BKN', 'BK')
    elif 'BKS' in district:
        district = district.replace('BKS', 'BK')
    elif 'QE' in district:
        district = district.replace('QE', 'QN')
    elif 'QW' in district:
        district = district.replace('QW', 'QN')
    
    return district

In [ ]:
def replace_districts(x):
  dictionary = {'Bronx': "BX",
           'Brooklyn': "BK",
           'Manhattan': "MN",
           'Queens': "QN",
           'Richmond': "SI",
           'Staten': "SI",
           }
  for key in dictionary.keys():
    x=x.replace(key, dictionary[key])
  return x


A function for load all of our datasets and merge them

In [ ]:
def load_data():
    #df=pd.read_csv('recycling.csv')
    df_recycling = pd.read_csv('https://data.cityofnewyork.us/resource/gaq9-z3hz.csv')
    df_recycling['district'] = df_recycling['district'].apply(clean_district)
    df_recycling['value'] = df_recycling['capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100']

    df_demographics = pd.read_csv('https://data.cityofnewyork.us/resource/w3c6-35wg.csv')
    df_demographics['district']=df_demographics.jurisdiction_name.apply(lambda x: replace_districts(x.split()[0])+x[-2:])
    demographics_df = df_demographics[['district','percent_female','percent_male','percent_permanent_resident_alien','percent_us_citizen','percent_receives_public_assistance']]

    df_populations = pd.read_csv('https://data.cityofnewyork.us/resource/xi7c-iiu2.csv')
    df_populations['names'] = df_populations['borough'] +' '+ str(0) + df_populations['cd_number'].astype(str)
    df_populations['district']=df_populations.names.apply(lambda x: replace_districts(x.split()[0])+x[-2:])
    populations_df = df_populations[['district','_1970_population','_1980_population','_1990_population','_2000_population','_2010_population']]

    df_social = pd.read_excel('https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/soc_2019_acs5yr_cdta.xlsx')
    df_social['No_diploma_percentage'] = 100*df_social['EA_9t12NDE'] / df_social['EA_P25plE']
    social_df = df_social[['GeoID','HHCompP','NtvNYSP','NtvNotNYSP','EA_BchDHP','No_diploma_percentage','EA_LTHSGrP']]
    social_df.columns=['GeoID','with_computer','born_nyc','not_born_nyc','bachelors_deg','No_diploma_percentage','less_than_high_school']

    df_econ = pd.read_excel('https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/econ_2019_acs5yr_cdta.xlsx')
    df_econ['less_than_10k'] = 100*df_econ['HHIU10E'] / df_econ['HH2E']
    df_econ['More_than_200k'] = 100*df_econ['HHI200plE'] / df_econ['HH2E']
    econ_df = df_econ[['GeoID','CvEm16pl1P','CvLFUEm1P','FamIU10P','MdHHIncC','less_than_10k','More_than_200k']]
    econ_df.columns=['GeoID','employed','unemployed','number_rooms_median','income_median','less_than_10k','More_than_200k']

    df_housing = pd.read_excel('https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/hous_2019_acs5yr_cdta.xlsx')
    df_housing['less_than_200k'] = 100*(df_housing['VlU50E'] + df_housing['Vl50t99E'] + df_housing['Vl100t149E'] +df_housing['Vl150t199E'] )/ df_housing['OOcHU2E']
    df_housing['More_than_1m'] = 100*df_housing['Vl1milplE'] / df_housing['OOcHU2E']

    housing_df=df_housing[['GeoID','MdVlC','MdGRC','MdRmsC','less_than_200k','More_than_1m']]
    housing_df.columns=['GeoID','value_median','gross_rent_median','number_rooms_median','less_than_200k','More_than_1m']

    df_demo = pd.read_excel('https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/demo_2019_acs5yr_cdta.xlsx')
    demo_df =df_demo[['GeoID','MdAgeC','PopU5P','Pop5t9P','Pop10t14P','Pop15t19P','Pop20t24P','Pop25t29P','Pop30t34P','Pop35t39P','Pop40t44P','Pop45t49P','Pop50t54P','Pop55t59P','Pop60t64P','Pop65t69P','Pop70t74P','Pop75t79P','Pop80t84P','Pop85plP','PopU181P','Pop65pl1P','Hsp1P','BlNHP','AsnNHP']]



    demo_df["Age_0_19"] = demo_df['PopU5P'] + demo_df['Pop5t9P'] + demo_df['Pop10t14P'] + demo_df['Pop15t19P']
    demo_df.drop(['PopU5P','Pop5t9P','Pop10t14P','Pop15t19P'],axis=1,inplace=True)

    demo_df["Age_20_39"] = demo_df['Pop20t24P'] + demo_df['Pop25t29P'] + demo_df['Pop30t34P'] + demo_df['Pop35t39P']
    demo_df.drop(['Pop20t24P','Pop25t29P','Pop30t34P','Pop35t39P'],axis=1,inplace=True)

    demo_df["Age_40_64"] = demo_df['Pop40t44P'] + demo_df['Pop45t49P'] + demo_df['Pop50t54P'] + demo_df['Pop55t59P'] + demo_df["Pop60t64P"]
    demo_df.drop(['Pop40t44P','Pop45t49P','Pop50t54P','Pop55t59P',"Pop60t64P"],axis=1,inplace=True)

    demo_df["Age_65_74"] = demo_df['Pop65t69P'] + demo_df['Pop70t74P'] 
    demo_df.drop(['Pop65t69P','Pop70t74P'],axis=1,inplace=True)

    demo_df["Age_75_over"] = demo_df['Pop75t79P'] + demo_df['Pop80t84P'] + demo_df['Pop85plP'] 
    demo_df.drop(['Pop75t79P','Pop80t84P','Pop85plP'],axis=1,inplace=True)

    #merged_df1 = df.merge(df_recycling, left_on='district', right_on='district',how='inner')
    merged_df1 = df_recycling
    merged_df2 = merged_df1.merge(demographics_df, left_on='district', right_on='district',how='inner')
    merged_df3 = merged_df2.merge(populations_df, left_on='district', right_on='district',how='inner')
    merged_df4 = merged_df3.merge(demo_df, left_on='district', right_on='GeoID',how='inner')
    merged_df5 = merged_df4.merge(social_df, left_on='GeoID', right_on='GeoID',how='inner')
    merged_df6 = merged_df5.merge(econ_df, left_on='GeoID', right_on='GeoID',how='inner')
    merged_df = merged_df6.merge(housing_df, left_on='GeoID', right_on='GeoID',how='inner')
    merged_df.drop('number_rooms_median_y',axis=1,inplace=True)
    merged_df.drop_duplicates('district', inplace = True) 
    merged_df = merged_df.reset_index(drop=True)

    df = merged_df
    #for later
    df_bokeh = merged_df.copy()

    df.dropna(inplace=True)
    df.drop(['district','GeoID','month_name','fiscal_month_number','fiscal_year','diversion_rate_total_total_recycling_total_waste_','capture_rate_paper_total_paper_max_paper_','capture_rate_mgp_total_mgp_max_mgp_','capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100'], axis=1, inplace=True)

    return df,df_bokeh


#### Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.

Since we have more than one dataset we are going to load and analyze one dataset at a time. Intuitive plots will also be presented to make everyone familiar with our datasets.



#### Recycling rates by district

We load the recycling [rates by community district dataset](https://data.cityofnewyork.us/Environment/Recycling-Diversion-and-Capture-Rates/gaq9-z3hz) and then we inspect the variables, to have an overview of the information contained in the dataset. 

For each Community District, its Recycling Diversion rate (percentage of total municipal solid waste collected by the Department of Sanitation that is disposed of by recycling) and Capture Rate (percentage of total Paper or Metal/Glass/Plastic in the waste stream that is disposed of by recycling).

Capture rate is the amount of materials set out for residential recycling collection as a percentage of designated recyclable materials in both recycling and refuse streams. This ratio measures how much of the targeted materials are actually being recycled, which is a measure of how successfully such materials are recycled.

Lets see all the variable this dataset is providing us:

- **Zone** : Different zones (text)

- **District** : One of NYC's 59 community districts which correspond to Sanitation districts. (text)

- **Fiscal Month Number** : The number of the fiscal month. The NYC fiscal year starts in July so July is fiscal month 1. (int)

- **Fiscal Year** : Year that corresponds to NYC fiscal year. The NYC fiscal year starts in July and ends in June. (int)

- **Month Name** : Name of the month (text)

- **Diversion Rate-Total** : Total Recycling / Total Waste	

- **Capture Rate-Paper** : Total Recyclable Paper / Max Recyclable Paper in Waste Stream

- **Capture Rate-MGP** : Total Recyclable Metal, Glass, Plastic & Beverage Cartons / Max Recyclable Metal, Glass, Plastic & Beverage Cartons in Waste Stream

- **Capture Rate-Total** :  ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100



In [ ]:
df_recycling = pd.read_csv('https://data.cityofnewyork.us/resource/gaq9-z3hz.csv')

In [ ]:
df_recycling['district'] = df_recycling['district'].apply(clean_district)

In [ ]:
df_recycling.head(5)

In [ ]:
round(df_recycling.describe(),2)

Above us is a table with some basic sttistics for one of our main dataset. We can see that the max value of recycling percentage is 75% when the min value in the same variable is 21%. We can also observe that the percentages in the paper recycling are smaler that the one in the Metal, Glass, Plastic & Beverage Cartons. The next plot illustrates this comparison between paper and the other per district. 

In [ ]:
#fig, ax = plt.subplots(figsize=(16,6))
fig = plt.figure(figsize=(22,8))
ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.


df_recycling \
    .groupby('district') \
    .mean() \
    .plot(kind='bar', y='capture_rate_paper_total_paper_max_paper_', ax=ax,
         title='The total recycle rates for Metal, Glass, Plastic & Beverage Cartons compared with the ones for Paper per district',position=0,width=0.4,color='red')

df_recycling \
    .groupby('district') \
    .mean() \
    .plot(kind='bar', y='capture_rate_mgp_total_mgp_max_mgp_', ax=ax2,
         position=1,width=0.4,color='Blue')

plt.show()

When we see the plot above, its more clear that in general the Metal, Glass, Plastic & Beverage Cartons group has higher rates but in many districts the paper overcomes it and be in top. Examples of that are the districs BK01 and MN09. In the plot below we show the rates for the compination of the two previous categories.  

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
df_recycling \
    .groupby('district') \
    .mean() \
    .plot(kind='bar', y='capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100', ax=ax,
         title='The total recycle rates for all the materials (100%)')
plt.show()

From the total rates we can see that the district with the biggest is QN11 with BK10 folowing. The district with the smallest rate is BX01

The Total recycle rates per zone in the city of new york. Here we plot again the total rates this time per zone since the districts codes are not well knowed in the public. The district with the biggest rate belongs to Quens West where the BX01 to Bronx

In [ ]:
import random

month = df_recycling.fiscal_month_number.value_counts()

districts = df_recycling.district.unique().tolist()
zones = df_recycling._zone.unique().tolist()
#+del districts[-1]
districts.sort()

count=1
for zone in zones:
    fig, ax = plt.subplots(figsize=(16,6))
    
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    
    df = (df_recycling[(df_recycling['_zone'] == zone)]).reset_index()
    df\
    .groupby('district') \
    .mean() \
    .plot(kind='bar', y='capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100', ax=ax,
         title='The total recycle rates for all the materials (100%)',color = color)
    #ax = sns.barplot(x='fiscal_month_number',y='capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100',data=data)
    ax.set_ylabel('Total Recycle rates')
    plt.xticks(rotation=90)
    plt.title(zone)
    plt.tight_layout()
    plt.show()



#### Community districts in new york

This dataset [(link)](https://data.cityofnewyork.us/City-Government/DSNY-Districts/i6mn-amj2) conteins the lat,log of each New York destrict as long with some usefull information about them. In order to have a better overview of this dataset lets analyze its variables in detail : 

- **DISTRICT** : District abbreviation as defined by DSNY

- **DISTRICTCODE** : District Code as defined by DCP

- **FID** : In ArcGIS, the FID is a system-managed value that uniquely identifies a record or feature. 

- **GlobalID** : The GlobalID is automatically assigned by ArcGIS software when a row is created.

- **SHAPE_Area** : square footage of the polygon

- **SHAPE_Length** : length in feet of the geographic feature

- **multipolygon** :	shape of the geographic feature




We will first have to clean our dataset in the format the other districts we have are.

In [ ]:
df_districts = pd.read_csv('https://data.cityofnewyork.us/api/views/i6mn-amj2/rows.csv?accessType=DOWNLOAD&bom=true&format=true')

In [ ]:
df_districts['DISTRICT'] = df_districts['DISTRICT'].apply(clean_district)

In the figure above we can see a plot illustrating the square foot of each district. This will help us understand the how big some districts are compared to the others.

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Square footage of each area")
ax = sns.barplot(y='SHAPE_Area', x='DISTRICT',data=df_districts)
plt.xticks(rotation=70)
plt.tight_layout()

Next we are going to present each one of the New York districts on map for better understanding

In [ ]:
with urlopen('https://data.cityofnewyork.us/resource/i6mn-amj2.geojson') as response:
    geo_json = json.load(response)

In [ ]:
for feature in geo_json['features']:
    feature['properties']['district'] = clean_district(feature['properties']['district'])

In [ ]:
geo_data = df_recycling \
    .groupby('district') \
    .mean() \
    .reset_index(level=0) \
    [['district', 'capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100']] \
    .rename(columns={'capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100': 'value'})

In [ ]:
recycling_fig = px.choropleth_mapbox(
    geo_data,
    geojson=geo_json,
    locations='district',
    color='value',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    title = 'The 59 different districts of New York and their recycling rates',
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'value':'Recycling %'}
)
recycling_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
recycling_fig.show()

These are the districts of New York colored by the percentage of recycling. The plot is illustrated below 

#### Locations of recycling bins in New York City

This dataset [(link)](https://data.cityofnewyork.us/Environment/Public-Recycling-Bins/sxx4-xhzg) contains the Locations of public recycling bins throughout NYC.

- **Borough** : The Borough name

- **Site type**	: Where the bin is located in the area

- **Park/Site Name** : The name of the street

- **Address** : The full address

- **Latitude**	

- **Longitude**



In [ ]:
df_bins = pd.read_csv('https://data.cityofnewyork.us/resource/sxx4-xhzg.csv')

In [ ]:
recycling_fig.add_scattermapbox(
    lat=df_bins['latitude'],
    lon=df_bins['longitude'],
    mode = 'markers'
)
recycling_fig.show()

Using the location of the bins can genarates some new features for our analysis and Ml later

#### Community districts populations

This dataset is used for our regrassion later. Its a simple dataset containing the population information of the destricts from 1970-2010

https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2

In [ ]:
df_populations = pd.read_csv('https://data.cityofnewyork.us/resource/xi7c-iiu2.csv')

In [ ]:
df_recycling['district'] = df_recycling['district'].apply(clean_district)

In [ ]:
df_populations.head()

#### Demographic, Social, Economic, and Housing Profiles by Community District

In this section we are going to investigate different Demographic, Social, Economic, and Housing statistics that it could may affect the recycling rates per destrict. This [link](https://www1.nyc.gov/site/planning/planning-level/nyc-population/american-community-survey.page.page) host the collection of all this different datasets we are going to use for the analysis.

##### Demographic profile

This Excel file is from a 2015-2019 ACS database [link](www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/demo_2019_acs5yr_cdta.xlsx) . For each of the 59 destricts it contains 106 different variables so we are not going to explain each one of them but we will still present them as categories. 

The first category is present data about the Sex of the total population as well as all the different Age groups.
The second one consist of info regarding the Race and origin of the population. 
Last we have two more categories analizyng the exact origin of the Spanish and the Asian population.
We are not going to use all of these variables in our analisys since a lot of them doesnt provide us with any valuable informations.


In [ ]:
df_demo = pd.read_excel(
    'https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/demo_2019_acs5yr_cdta.xlsx',
)
df_demo = df_demo[df_demo['GeoID'].isin(df_recycling['district'])]

This figure presents the Age distribution of each district coloring with yellow color all the districts with high median age and with dark blue the ones with the lowest median age.

In [ ]:
med_age_fig = px.choropleth_mapbox(
    df_demo,
    geojson=geo_json,
    locations='GeoID',
    color='MdAgeE',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'MdAgeE':'Median age'}
)
med_age_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
med_age_fig.show()

##### Social profile

This Excel file is from a 2015-2019 ACS database [link](https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/soc_2019_acs5yr_cdta.xlsx) . For each of the destricts it contains 442 different variables. The social charecteristic this dataset contains in many cases are irrelevant with the focus of this project. We will focus on the educational information this dataset provides and we will analyze any conection these can have with the recycling rates. 

In [ ]:
df_social = pd.read_excel(
    'https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/soc_2019_acs5yr_cdta.xlsx',
)
df_social = df_social[df_social['GeoID'].isin(df_recycling['district'])]

In [ ]:
df_social['EA_BchDHE_percentage'] = 100*df_social['EA_BchDHE'] / df_social['EA_P25plE']
bachelor_fig = px.choropleth_mapbox(
    df_social,
    geojson=geo_json,
    locations='GeoID',
    color='EA_BchDHE_percentage',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'EA_BchDHE_percentage':"Percentage with Bachelor's degree" },
    title = "4"
)
bachelor_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
bachelor_fig.show()

In the figure above we illustrate the percentage of people having a bachelor degreee

In [ ]:
df_social['EA_BchDHE_percentage'] = 100*df_social['EA_9t12NDE'] / df_social['EA_P25plE']
bachelor_fig = px.choropleth_mapbox(
    df_social,
    geojson=geo_json,
    locations='GeoID',
    color='EA_BchDHE_percentage',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'EA_BchDHE_percentage':"Percentage of people finish 9th-12th grade with no diploma" }
)
bachelor_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
bachelor_fig.show()

In the figure above we illustrate the percentage of people that have finish 9th-12th grade but they dont have any diploma

In [ ]:

df_social['EA_BchDHE_percentage'] = 100*df_social['EA_LTHSGrE'] / df_social['EA_P25plE']
bachelor_fig = px.choropleth_mapbox(
    df_social,
    geojson=geo_json,
    locations='GeoID',
    color='EA_BchDHE_percentage',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'EA_BchDHE_percentage':"Less than high school graduate" }
)
bachelor_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
bachelor_fig.show()

The last plot above show us the percentage of people that they havent finish highscool 

We can get from all the above plots that the center MN districts have the most educated people, while on the oter hand the least educated leave in BX district.

##### Economic profile

As we said before also this dataset contains many different columns with different economic aspects for the districts. We are basicaly care about the income and benfit category. The variables in this category are aplaining the basic income of the tatal households for each district. The dataset can be downloaded here [link]( https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/econ_2019_acs5yr_cdta.xlsx)

In [ ]:
df_econ = pd.read_excel(
    'https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/econ_2019_acs5yr_cdta.xlsx',
)
df_econ = df_econ[df_econ['GeoID'].isin(df_recycling['district'])]

In the next three maps we illustrate the Median household income per district and two other variable show the amount of households with income lower than 10,000 and more than 200,000. More than 90% of the districts dont have an income more that 200k. Only in the center of the NY you can find a few districts with that amount of house income. BK05 seems the district with the most people living with less than 10k.Where on the other hand MN07-8 is one of the destrict with the most rich people. 

In [ ]:
#Should we also make them % and not total number like before?

income_fig = px.choropleth_mapbox(
    df_econ,
    geojson=geo_json,
    locations='GeoID',
    color='MdHHIncE',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'MdHHIncE':"Median household income (USD)" }
)
income_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
income_fig.show()

In [ ]:


income_fig = px.choropleth_mapbox(
    df_econ,
    geojson=geo_json,
    locations='GeoID',
    color='HHIU10E',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'HHIU10E':"Household income of less than 10,000(USD)" }
)
income_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
income_fig.show()

In [ ]:
income_fig = px.choropleth_mapbox(
    df_econ,
    geojson=geo_json,
    locations='GeoID',
    color='HHI200plE',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'HHI200plE':"Income :$200,000 or more (USD)" }
)
income_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
income_fig.show()

##### Housing profile

Last we have information about the NY households. Again we are only going to use variables regarding the number of rooms and the total value of the houses and not the hole dataset. The dataset can be obtain by the following [link](https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/hous_2019_acs5yr_cdta.xlsx) 

In [ ]:
df_housing = pd.read_excel(
    'https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/hous_2019_acs5yr_cdta.xlsx',
)
df_housing = df_housing[df_housing['GeoID'].isin(df_recycling['district'])]

In [ ]:
df_housing.groupby('GeoID')

In [ ]:
rooms_fig = px.choropleth_mapbox(
    df_housing,
    geojson=geo_json,
    locations='GeoID',
    color='MdRmsE',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'MdRmsE':"Median number of rooms" }
)
rooms_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
rooms_fig.show()

In [ ]:
rooms_fig = px.choropleth_mapbox(
    df_housing,
    geojson=geo_json,
    locations='GeoID',
    color='MdVlE',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9.25, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    labels={'MdVlE':"Median Value of the houses" }
)
rooms_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
rooms_fig.show()

As we expected the city centered seams to have the most expensives houses with the smallest amount of rooms at the same time. The exact oposite trend can be found in the suburbs where the most cheap area to buy a house is in BX districts. 

In [ ]:
# Social-economic data frames
data_se = [df_demo, df_social, df_econ, df_housing]
df_se = reduce(lambda  left,right: pd.merge(left,right, on='GeoID'), data_se)
recycling_avg = df_recycling.groupby('district').mean()

The vast majority of our interactive visualizations are derived from [NYC OpenData](https://data.cityofnewyork.us/) sources. As a part of our visualization, we demonstrate how we prepared the data for plotting. We export a geojson file in order to accomplish this. Each row represents one of the five boroughs: Brooklyn (Kings County), Queens (Queens County), Manhattan (New York County), the Bronx (Bronx County), and Staten Island (Richmond County). Detailed representations of each district are presented in the rows along with the data that will be plotted on the map. Geopandas dataframe is used to store districts' names and polygons. To create a plot, we merge it with the dataframe containing the rest of the data.

### Visualizations.


In this Part we will present our main interactive plots that we also used in our website. These plots are a lot more complex and they are going to help us find all the paterns we need to form a worth telling story. We did not include the output in this parts since all the plots above can be found in our webpage. The notebook was too heavy to uploaded if we kept the plots here.


The first visualization is like the map we had before but this time we present with the same color the zones of each district and we have also a buttom to choose whether or not to show the recycling bins.

In [ ]:
geo_data['dummy_value'] = np.arange(len(geo_data))
district_fig = px.choropleth_mapbox(
    geo_data,
    geojson=geo_json,
    locations='district',
    color='dummy_value',
    featureidkey='properties.district',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=9, center = {"lat": 40.72, "lon": -73.935242},
    opacity=0.5,
    hover_data = {'dummy_value':False},
    labels={'district': 'District'},
)

district_fig.update_layout(
    coloraxis_showscale=False,
    width=700,
    height=450,
    margin={"r":0,"t":0,"l":0,"b":0})

district_fig.add_scattermapbox(
    lat=df_bins['latitude'],
    lon=df_bins['longitude'],
    mode = 'markers',
    hoverinfo='skip',
    visible=False
)

bin_button = dict(
    label='Show recycling bins',
    method='restyle',
    args=[{'visible': True}, [1]],
    args2=[{'visible': False}, [1]])

# Create menus
district_fig.update_layout(
    updatemenus=[
        dict(type='buttons', active=-1, y=1, x=0, xanchor='left', buttons=[bin_button])]
)
#district_fig.write_html("socialdata22-website/content/homepage/00_districts.html")
district_fig.show()

The second interactive plot presents diferent socialeconomic factors that the user can choose and see the impact they have in each district. Each time the user choose one factor the map is coroled based on it. The choise of showing or not the recycling bins is still there.

In [ ]:
import plotly.graph_objects as go
df_se = df_se.sort_values(by='GeoID')
recycling_avg = recycling_avg.sort_values(by='district')

# Map keyword arguments
map_kwargs = dict(
    geojson=geo_json,
    featureidkey='properties.district',
    locations=df_se.GeoID,
    colorscale='Viridis',
    hovertemplate = 'District: %{location}<br>%{meta[0]}: %{z:.1f}<extra></extra>',
    colorbar_title = '%{meta[0]}',
    marker_opacity=0.5,
    marker_line_width=0,
)

# Layout keyword arguments
layout_kwargs = dict(
    title={
        'text': 'Socio-economic factors and recycling behaviour by district',
        'y': 0.975,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    mapbox_style='carto-positron',
    mapbox_zoom=9,
    mapbox_center = {'lat': 40.72, 'lon': -73.935242},
    margin={'r':0,'t':35,'l':182.5,'b':0},
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

# Social-economic factors
se_variables = [
    'MdAgeE', 'EA_BchDHE_percentage', 'MdHHIncE','MdRmsE']

se_names = [
    'Median age', "Percentage with<br>Bachelor's degree",
    'Median household<br>income (USD)',  'Median number<br>of rooms']

n_se = len(se_variables)
se_trace_idx = [i for i in range(n_se)]

# Recycling variables
rc_variables = [
    'capture_rate_paper_total_paper_max_paper_',
    'capture_rate_mgp_total_mgp_max_mgp_',
    'capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100']

rc_names = [
    'Avg. paper<br>recycling rate',
    'Avg. metal, glass,<br>plastic &<br>beverage cartons<br>recycling rate',
    'Avg. total<br>recycling rate']
n_rc = len(rc_variables)
rc_trace_idx = [n_se + i for i in range(n_rc)]

# All variables
variables = se_variables + rc_variables
names = se_names + rc_names
trace_idx = se_trace_idx + rc_trace_idx
n = n_se + n_rc

# Create maps and buttons
fig = go.Figure()
buttons = list()
for i, variable in enumerate(variables):
    visible = True if i == 0 else False
    
    if i in rc_trace_idx:
        df = recycling_avg
    else:
        df = df_se
    
    fig.add_trace(
        go.Choroplethmapbox(
            z=df[variable],
            meta=[names[i]],
            visible=visible,
            **map_kwargs
        )
    )
    visible = [True if j == i else False for j in trace_idx]
    buttons.append(
        dict(label=names[i], method='restyle', args=[{'visible': visible}, trace_idx])
    )

# Recycling bins
fig.add_trace(
    go.Scattermapbox(
        lat=df_bins['latitude'],
        lon=df_bins['longitude'],
        mode = 'markers',
        visible=False,
        opacity=0.5,
        hoverinfo='skip',
        name='Recycling bins',
        showlegend=True)
)

# Button for recycling bins
bin_trace_idx = n
bin_button = dict(
    label='Show recycling bins',
    method='restyle',
    args=[{'visible': True}, [bin_trace_idx]],
    args2=[{'visible': False}, [bin_trace_idx]])

# Create menus
fig.update_layout(
    updatemenus=[
        dict(type='buttons', active=-1, y=1, x=-0.225, xanchor='left', buttons=[bin_button]),
        dict(type='buttons', active=0, y=0.85, x=-0.225, xanchor='left', buttons=buttons)]
)

# Add text annotations
fig.add_annotation(
            y=0.84,
            x=-0.25,
            textangle=270,
            yanchor='top',
            xanchor='right',
            xref='paper',
            yref='paper',
            showarrow=False,
            text='Socio-economic factors')

fig.add_annotation(
            y=0.52,
            x=-0.25,
            textangle=270,
            yanchor='top',
            xanchor='right',
            xref='paper',
            yref='paper',
            showarrow=False,
            text='Recycling variables')

fig.update_layout(**layout_kwargs)
fig.show()
#fig.write_html("socio_economic_map.html")

The next plot shows how the recycling rates for Paper , Others and total have changed over the period of 2018-2019 that we had available data. The user can choose between one of these categories and see the change.  

In [ ]:
df_recycling['year-month'] = (
    df_recycling.fiscal_year.astype(str) + '-' +  df_recycling.fiscal_month_number.map('{:02d}'.format) )
df_recycling['datetime'] = pd.to_datetime(df_recycling['year-month'])
df_recycling = df_recycling.sort_values('datetime', ascending=True)

df_district = pd.DataFrame(
    {'district': np.sort(df_recycling.district.unique())}
)

# Map keyword arguments
map_kwargs = dict(
    geojson=geo_json,
    featureidkey='properties.district',
    locations=df_district.district,
    colorscale='Viridis',
    hovertemplate = 'District: %{location}<br>%{meta[0]}: %{z:.1f}<extra></extra>',
    colorbar_title = '%{meta[0]}',
    marker_opacity=0.5,
    marker_line_width=0,
)

# Layout keyword arguments
layout_kwargs = dict(
    title={
        'text': 'Recycling behaviour over time',
        'y': 0.975,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    mapbox_style="carto-positron",
    mapbox_zoom=9,
    mapbox_center = {'lat': 40.72, 'lon': -73.935242},
    margin={'r':0,'t':35,'l':182.5,'b':100},
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

# Recycling variables
rc_variables = [
    'capture_rate_paper_total_paper_max_paper_',
    'capture_rate_mgp_total_mgp_max_mgp_',
    'capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100']

rc_names = [
    'Paper<br>recycling rate',
    'Metal, glass,<br>plastic &<br>beverage cartons<br>recycling rate',
    'Total<br>recycling rate']
n_rc = len(rc_variables)

fig = go.Figure()

# Timepoints in the data
tps = df_recycling['year-month'].unique()

# Create trace for each variable and each time point
for i, variable in enumerate(rc_variables):
    for tp in tps:
        # Make sure that values matches order of districts
        df_z = df_recycling[df_recycling['year-month'] == tp].sort_values(by='district')
        df_z = df_district.merge(df_z, on='district', how='left')
        fig.add_trace(
            go.Choroplethmapbox(
                z=df_z[variable],
                zmin = df_recycling[variable].min(),
                zmax = df_recycling[variable].max(),
                visible=False,
                meta=[rc_names[i]],
                **map_kwargs
            ))

# Make first trace visible
fig.data[0].visible = True

# Create list of steps for each variable
steps = {variable: [] for variable in rc_variables}

# Make steps
for i, variable in enumerate(rc_variables):
    for j in range(len(tps)):
        step = dict(
            label=tps[j],
            method='restyle',
            args=[{'visible': [False] * len(fig.data)}],
        )
        step['args'][0]['visible'][j + i*len(tps)] = True  # Toggle i'th trace to visible
        steps[variable].append(step)

# Create sliders
sliders = list()
for variable in rc_variables:
    sliders.append([dict(active=0, currentvalue={'prefix': 'Year-Month: '}, steps=steps[variable])])
    
# Create buttons
buttons = list()
for i in range(n_rc):
    buttons.append(
         dict(label=rc_names[i], method='update', args=[{"sliders": sliders[i]},{"sliders": sliders[i]}, [i]])
    )

fig.update_layout(
    updatemenus=[
        dict(type='buttons', active=0, y=1, x=-0.225, xanchor='left', buttons=buttons)]
)

fig.update_layout(
    sliders=sliders[0],
    **layout_kwargs
)
fig.show()
#fig.write_html("recycling_map.html")

In the next plot we tried to see how different factors that we have already analyze affect the recycling rate. From the next plot the user can choose a category: Education,Econimics or Housing and compare the factors presented there with recycling. For example we can see that in districts with high percentage of bachelor degrees the recycling rates are also high. It also seems that in destricts like BX01, which is the lowest recycling percentage, the bachelor rates are too small and the uneducated people are many. 

In [ ]:
from bokeh.layouts import layout
from bokeh.models import HoverTool
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import output_notebook, curdoc
from bokeh.models import ColumnDataSource, Legend, FactorRange
from bokeh.transform import dodge

output_notebook()

df,df_bokeh = load_data()
df = df_bokeh
df.dropna(inplace=True)

# Convert dataframe to ColumnDataSource
src = ColumnDataSource(df)

# Create empty figures
p_education = figure(
    plot_width=1200,
    plot_height=700,
    title='Recycling Rates by Education factors',
    x_axis_label='Districts',
    y_axis_label='Rates',
    x_range=df['district']) 

p_economic = figure(
    plot_width=1200,
    plot_height=700,
    title='Recycling Rates by Economic factors',
    x_axis_label='Districts',
    y_axis_label='Rates',
    x_range=df['district'])

p_housing = figure(
    plot_width=1200,
    plot_height=700,
    title='Recycling Rates by Housing factors',
    x_axis_label='Districts',
    y_axis_label='Rates',
    x_range=df['district'])

# Define the columns to use for each bar
bar_cols_education = ['value', 'bachelors_deg','No_diploma_percentage']
legend_labels_education = ['Recycling rate', 'Bachelor degree %',"Finsh high school with no other diploma"]

# List for legend items 
items_education = []
items_economic = []
items_housing = []

bars_education = {}
bars_economic = {}
bars_housing = {}
x = -0.25
for idx, col in enumerate(bar_cols_education):
    bars_education[col] = p_education.vbar(
        x=dodge('district', x, range=p_education.x_range),#'district',
        top=col,
        source=src, 
        legend_label=legend_labels_education[idx],
        color=Spectral6[idx],
        width=0.2,
        muted = True)
    items_education.append((legend_labels_education[idx], [bars_education[col]]))
    x = x+0.25
p_education.xaxis.axis_label_standoff = 250    

    
bar_cols_economic = ['value', 'less_than_10k','More_than_200k']
legend_labels_economic = ['Recycling rate', 'Less than 10k $',"More than 200k $"]

x = -0.25

for idx, col in enumerate(bar_cols_economic):    
    bars_economic[col] = p_economic.vbar(
        x=dodge('district', x, range=p_economic.x_range),
        top=col,
        source=src, 
        legend_label=legend_labels_economic[idx],
        color=Spectral6[idx],
        width=0.2,
        muted = True)
    items_economic.append((legend_labels_economic[idx], [bars_economic[col]]))
    x = x+0.25
p_economic.xaxis.axis_label_standoff = 250    

bar_cols_housing = ['value', 'less_than_200k','More_than_1m']
legend_labels_housing = ['Recycling rate', 'House Cost less than 200k ($)','House Cost more than 1M($)']

x = -0.25

for idx, col in enumerate(bar_cols_housing):    
    bars_housing[col] = p_housing.vbar(
        x=dodge('district', x, range=p_housing.x_range),
        top=col,
        source=src, 
        legend_label=legend_labels_housing[idx],
        color=Spectral6[idx],
        width=0.2,
        muted = True)
    items_housing.append((legend_labels_housing[idx], [bars_housing[col]]))
    x = x+0.25

p_housing.xaxis.axis_label_standoff = 250    

p_education.legend.visible = False
p_economic.legend.visible = False
p_housing.legend.visible = False

legend_education = Legend(items=items_education, location='top_left')
legend_economic = Legend(items=items_economic, location='top_left')
legend_housing = Legend(items=items_housing, location='top_left')
p_education.add_layout(legend_education, 'left')
p_economic.add_layout(legend_economic, 'left')
p_housing.add_layout(legend_housing, 'left')

p_education.legend.click_policy = "mute" #you can also try "hide"
p_economic.legend.click_policy = "mute"
p_housing.legend.click_policy = "mute"

p_education.add_tools(HoverTool(
    tooltips=[
        ("Recycling rates", "@value{0.00}"),
        ("Percentage of citizens with bechelor degree", "@bachelors_deg{0.00}"),
        ("Percentage of citizens with no degree", "@No_diploma_percentage{0.00}")],
    renderers=[bars_education['value']]))
p_economic.add_tools(HoverTool(
    tooltips=[
        ("Recycling rates", "@value{0.00}"),
        ("Houses with icnome more than 200k", "@More_than_200k{0.00}"),
        ("Houses with icnome less than 10k", "@less_than_10k{0.00}")],
    renderers=[bars_economic['value']]))
p_housing.add_tools(HoverTool(
    tooltips=[
        ("Recycling rates", "@value{0.00}"),
        ("Houses with cost less than 200k", "@less_than_200k{0.00}"),
        ("Houses with cost more than 1M", "@More_than_1m{0.00}")],
    renderers=[bars_housing['value']]))

p_education.xaxis.major_label_orientation = 1.2#"vertical"
p_economic.xaxis.major_label_orientation = 1.2#"vertical"
p_housing.xaxis.major_label_orientation = 1.2#"vertical"


l1 = layout([[p_education]])
l2 = layout([[p_economic]])
l3 = layout([[p_housing]])

tab1 = Panel(child=l1, title="Education")
tab2 = Panel(child=l2, title="Economics")
tab3 = Panel(child=l3, title="Housing")
tabs = Tabs(tabs=[tab1, tab2, tab3])

curdoc().add_root(tabs)
show(tabs)

In our last plot the user is able to see the areas which their distance is closer to a recycling bin. With this vizualization we want to show that there are some areas with limited access to a bin and that is something that can also affect the recycling rates of that area.

In [ ]:
import zipfile
from io import BytesIO
import requests

url = 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip'
r = requests.get(url)
buf1 = BytesIO(r.content)
with zipfile.ZipFile(buf1, "r") as f:
    for name in f.namelist():
        if name.endswith('.csv'):
            with f.open(name) as zd:
                df_pluto = pd.read_csv(zd, low_memory=False)
            break
            
#df_pluto = pd.read_csv('https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip', low_memory=False)
df_districts = pd.read_csv('https://data.cityofnewyork.us/api/views/i6mn-amj2/rows.csv?accessType=DOWNLOAD')[['DISTRICT', 'DISTRICTCODE']]
df_bins = pd.read_csv('https://data.cityofnewyork.us/resource/sxx4-xhzg.csv')

df_pluto = df_pluto.merge(df_districts, left_on='cd', right_on='DISTRICTCODE')
df_pluto = df_pluto[['DISTRICT','latitude', 'longitude', 'borough']]
df_pluto = df_pluto.dropna()
df_bins = df_bins[df_bins['latitude'].notna() & df_bins['longitude'].notna()]

# Compute coordinates in radians
df_pluto['latitude_rad'] = df_pluto['latitude'].map(radians)
df_pluto['longitude_rad'] = df_pluto['longitude'].map(radians)
df_bins['latitude_rad'] = df_bins['latitude'].map(radians)
df_bins['longitude_rad'] = df_bins['longitude'].map(radians)

# Compute pairwise haversine distances
distances = haversine_distances(
    df_pluto[['latitude_rad', 'longitude_rad']].to_numpy(),
    df_bins[['latitude_rad', 'longitude_rad']].to_numpy()
)
# Convert to km
distances = distances * 6371000/1000

# Distance to nearest bin
min_distances = distances.min(axis=1)

df_pluto['bin_distance'] = min_distances

 


In [ ]:
# Setup colormap
colors = ['#edf8fb','#b2e2e2','#66c2a4','#2ca25f','#006d2c']
colors.reverse()
vmin = 0
vmax = min_distances.max()
levels = len(colors)
cm = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

# The original data
x_orig = df_pluto.longitude.to_numpy()
y_orig = df_pluto.latitude.to_numpy()
z_orig = df_pluto.bin_distance.to_numpy()
# Make a grid
x_arr = np.linspace(np.min(x_orig), np.max(x_orig), 1000)
y_arr = np.linspace(np.min(y_orig), np.max(y_orig), 1000)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
 
# Grid the values
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')
 
# Create the contour
contourf = plt.contourf(
    x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
 
# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf)

# Set up the folium plot
geomap = folium.Map([40.72, -73.935242], zoom_start=10, tiles="cartodbpositron")
 
# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
        'fillColor': x['properties']['fill'],
        'opacity':   0.6,
    }).add_to(geomap)

# Add the colormap to the folium map
cm.caption = 'Distance to nearest public recycling bin (km)'
geomap.add_child(cm)

### Data Analysis


#### Using linear regression to predict recycling rates based on socioeconomic variables

Previously, we observed that socioeconomic factors for the districts may be helpful in predicting recycling rates in the city based on the correlation between recycling data and socioeconomic factors. This section examines the relationship between recycling rates and socioeconomic factors using a linear regression model. We will examine different regrassion models in order to predict the target class (Recycling rates) and also observe the feature importance of our models to see if we can gain usefull information.

The final DF we used for our analysis is a combination of different variables that we choosed to use from the others.

In [ ]:
df,df_bokeh = load_data()

#use the avarage bin distance we calculate before for our regrassion model
df['bin_distance'] = df_pluto['bin_distance']

In [ ]:
# get the dummies and store it in a variable
#dummies = pd.get_dummies(df._zone)
# Concatenate the dummies to original dataframe
#df = pd.concat([df, dummies], axis='columns')
 
# drop the values
df.drop(['_zone','with_computer',"percent_female","percent_male","unemployed",'_1970_population','_1980_population','_1990_population','Age_40_64','Age_20_39','_2000_population','Age_0_19','Age_65_74','Age_75_over'], axis='columns',inplace=True)

Here we can observe all the variables that we are going to investigate with our regrassion models

In [ ]:
df.info()

In [ ]:
import seaborn as sns

# Helper function for plotting feature importance
def plot_features(columns, importances, n=20):
    df = (pd.DataFrame({"features": columns,
                        "feature_importance": importances})
          .sort_values("feature_importance", ascending=False)
          .reset_index(drop=True))
    
    sns.barplot(x="feature_importance",
                y="features",
                data=df[:n],
                orient="h")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import math

targets = np.array(df.value)
df = df.drop(["value"], axis=1)
data = np.array(df)
scaler = preprocessing.StandardScaler()


X_train,X_test,y_train,y_test = train_test_split(data,targets,test_size=0.10,random_state=5)
scaler.fit(X_train)

regressor = GradientBoostingRegressor(learning_rate=0.1, n_estimators=50, verbose=1,random_state =5)
regressor.fit(X_train, y_train)


rmse = math.sqrt(mean_squared_error( regressor.predict(X_test), y_test ))
print(f'The final MSE for the regression model is {rmse:.2f}')

In [ ]:
from sklearn.inspection import permutation_importance
feature_importance = regressor.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, np.array(df.columns)[sorted_idx])
plt.title("Feature Importance (MDI)")

result = permutation_importance(
    regressor, data, targets, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(
    result.importances[sorted_idx].T,
    vert=False,
    labels=np.array(df.columns)[sorted_idx],
)
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

From the plot above we can see the feature importance for the regrassion model we trained. The findings are very intresting since we see that the model consider the income as the most valuable feature with the employment status and the avarage age to follow. Variables as the income and the distance from the recycling bins are also consider as we expected important while other variables ralates to the ethnicity of the people seams also valuable for the model. On the other hand the house value, the number of total population of the district as well as variables like the Percentage with diploma and the meian income seems to be irrelevant for the model.

All in all the model validate some of our initial thoughts about factors that can affect the recycling rates and also discovered some new like the employment status and their ethnicity.

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
df["value"] = targets
sns.heatmap(df.corr())

The plot here illustrates the correlation matrix of our dataset. We can also see from that that the variable less than 10k is neggative corelated with our target variable. This means that the more people with small income a district has the less the recycling rate will be. 

We also tried some more machine learning models to see if we could minimize the rmse. The GradientBoosting regrassion seems that performs the best. We cross validate all the models while we also make a grid search for the value of alphas for both Lasso and Ridge regrassion.

In [ ]:
from sklearn.linear_model import LassoCV

lasso_alphas = np.linspace(0, 10, 30)


reg = LassoCV(alphas=lasso_alphas, cv=15).fit(X_train, y_train)
rmse = math.sqrt(mean_squared_error( reg.predict(X_test), y_test ))
rmse

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

folds = KFold(n_splits = 10, shuffle = True, random_state = 100)
hyper_params = [{'alpha': np.linspace(0, 2, 40)}]

reg = Ridge()


model_cv = GridSearchCV(estimator = reg, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, y_train)    


In [ ]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head(5)

In [ ]:
# plotting cv results
plt.figure(figsize=(10,6))

plt.plot(cv_results["param_alpha"], cv_results["mean_test_score"])
plt.plot(cv_results["param_alpha"], cv_results["mean_train_score"])
plt.xlabel('alpha')
plt.ylabel('r-squared')
plt.title("Optimal Number of Alpha")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
# final model
alpha = 1

reg = Ridge(alpha=10).fit(X_train, y_train)
rmse = math.sqrt(mean_squared_error( reg.predict(X_test), y_test ))
rmse

For our last part of our analysis we will also try to see how a normal linear regrassion model performs and some plots we can have from that model. We will start by plotting the most important variables pairwise.

In [ ]:
# Get columns to keep and create new dataframe with those only
cols = ['employed','less_than_10k','born_nyc','percent_us_citizen','less_than_200k','Hsp1P','gross_rent_median','More_than_200k']
features = df[cols]
features['value'] = targets
sns.pairplot(features)

In [ ]:
from scipy import stats
#450 height
#700 pixels

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi


data = features.drop('value',axis=1)

fig = plt.figure(figsize=(15,20))
for i in range(len(data.columns)):
    fig.add_subplot(5, 3, i+1)
    ax = sns.regplot(x=df['value'], y = data[data.columns[i]], ci = 95)
    bbox_props = dict(boxstyle="round,pad=0.2", fc="white", ec='white', lw=0, alpha=0.6)
    ax.annotate("r = {:.2f}".format(pearsonr_ci(x=df['value'], y = data[data.columns[i]])[0]), xy=(.7, .9), xycoords=ax.transAxes, bbox=bbox_props)
    ax.annotate("p = {:.2f}".format(pearsonr_ci(x=df['value'], y = data[data.columns[i]])[1]), xy=(.7, .82), xycoords=ax.transAxes, bbox=bbox_props)
    plt.ylabel(data.columns[i])
    plt.subplots_adjust(wspace=0.4)
       
plt.show()

On the plot above we can see the Pearson correlation between our target variable and the 8 most rellevant features. We can clearly observe the possitive corralation in the employed variable and the neggative one in the less than 10k

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
# print the intercept
print(lm.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,df.drop('value',1).columns,columns=['Coefficient'])
coeff_df.reindex(coeff_df.Coefficient.abs().sort_values(ascending=False).index)

On the above plot we have the coeffitients of our simple regrassion model. We observe that the Median age variable has the highest neggative coeffitient. This basicaly means that the younger people recycle more than the oldest ones. But we cant take that as a grand truth since our model is too simple and with very few observation so its not that much trusted.

### Genre
#### Which genre of data story did you use?
For our major genre, we went with a partitioned poster. We also incorporated elements from other genres. For example, we utilized also the annotated charts to explain some main features of our data.

#### Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?

From the visual structuring category we mainly used the Consistent Visual Platform category as we had as our main vizualization a map which could change based on the users choise. From the highlighting category we used close-ups and zooming to give the user a focus on the NY districts and zones, where we wanted to guide to. We also used motion in the maps for over the years change of the recycling behaviors - giving the user a visual aid to see the different patterns. We lastly used feature distinction a lot in our interactive map providing the user a tool to distinguish between different socialeconomic factors or recycling behaviors. The transition guidance category did not apply to our project so none of the strategies from that category were used.


#### Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

From the ordering category we used user directed path, since you explore the website by scrolling down the site. We mainly direct the user through the story we want to tell while he is scrolling down the page when at the same time we are also giving the user freedom to explore for himself. From the interactive category we used stimulating default views, all of our maps were already zoomed in the NY area. We also used explicit instruction, giving the user instruction and suggestions on how to navigate to the visualizations. In addition to that we used filtering/selection/search for example in most of our maps and in the bar plot th user could filter the information he wanted to see. This was all done to guide the user through the visualization. Lastly from the messaging category we used introductory text at the top of the website to introduce the user to introduce the user to the problem and we also used the Summary/Synthesis category at the end to finish our story.

### Explain the visualizations you've chosen.


Several visualizations were developed to guide the viewer through a general exploration of the data sets, to a more detailed analysis of recycling behavior:

##### Bar Chart

One simple but important visualization tool. Provides an overview of the data as well as the reasoning behind the visuals. We made some bar chrats plots at the beggining of our analysis whitch helped us understand the data better. 

##### Interactive bar charts

A more advanced chart in which the user had the oportunity to choose different aspects of the problem and visualized them. This also gives you more freedom to explore without being overwhelmed by a lot of information because you can choose what you want to look at.

##### Interactive maps

Our most explainable plots due to the format of our data. We managed to create many different interactive maps for different variables regarding the NY. All of our data was district based so no other plot was really intresteed to investigate. With these maps we manage to explain our datasets find patterns and also make the user part of the analysis.


#### Why are they right for the story you want to tell?

Our initial goal was to investigate whether or not different social or economic factors affects the rates of recycling. As we mention before most of our data was geo data or based on each of the 59 NY districts. The use of maps to illustrate them seemed nesecessary and proved very helpful. In order to support these map plots we also choose to consider other simplier plots like bar plots. From these plots we had the chance to compare different values because we could present them at the same time with others. This was something that the map could not provide us. Overall, we believe that our plot choises were the best for both understanding the data as well as analyzing them.  

### Discussion. Think critically about your creation
#### What went well, what was missing and what could be improved.

We cleaned, explored, combined, and explained a number of data sets, as well as developed some interactive visualizations to summarize some of our main findings. The data set was first explored, and then the project scope was narrowed gradually, in order to create a website that conveyed a clear message.
The behavior of New York residents regarding recycling has been difficult to explain using socioeconomic factors since there is great variability in recycling rates across different districts, which cannot be explained by only socioeconomic factors. Even though we strongly believe that we end up with some usefull patterns. Factors like the educational level and the income per house seemed to follow the same trend with the recycling rates. All districts with more than 40% of their people having a bachelor degree also had recycling rates over 40%. Moreover, if we see the districts with the lowest rec rates they also have one of the highest uneducate percentage to their people and one of the lowest percentage of high educated at the same time. The same pattern follows the yearly income, where the lowest recycling rates districts also have the lowest yearly incomes. Our analysis didnt follow the same results in the avarage house price per district. In that factor although we have some results that can follow the upper trend in some districts, there are others that they dont validate them also. Another intresting pattern was that the avarage bin distance didnt affect allot the recycling rates. The only exception was the BX01 district which had the smallest recycling rates and big avarage bin distance. Apart from that, more that 85% of the districts had an avarage bin distance of less than 1 km which illustrates that NY city has done a good job providing recycling bins close to most of the houses.  

Recycling behaviors appear to be a much more complex phenomenon, which would require a great deal more information in order to fully explain. However, the analysis has revealed a number of interesting patterns. 

One of the thinks that it was diffecult for us to deal with was that we had access to a lot of data and variables. That may seems good for a data analysis but most of them was irrelevant with our problem the thing was that we didnt had any in advance knowlege to help us choose better the variables that we analysis. So we end up choosing with our own criteria this that means that we may have lost some information due to our decisions. 

#### Machine learning approach 
We have relied on a relatively large number of explainable variables in building the model explaining recycling behavior. 
Model improvement could be accomplished by adding additional features and removing irrelevant ones, which might contribute to explaining recycling behavior. In the project, we selected socioeconomic factors subjectively as the ones we found interesting, but we may have been able to make a more analytical choice out of the wide selection of factors found in the data sources.
Our main goal from our model is to make some prediction on the totall recycling rates that we have using all of the data we had in our analysis and also adding more. Moreover, we tried to see what was the feature importance based on ML models and see if this can lead us to some intresting results. At this point we have to admit that it was difficult to make a value model and thats because we only had access to 59 observations. We think though that our aproach was well established and it could have had more valueble results if we also had access to data from other cities. 

 
Overall, we were able to divide the work fairly equally among the group. Our team was able to acquire the necessary skills to implement those visualisations, and we are happy with the interactive and engaging results. The challenge has been to limit the content of visualisations in order to keep the user's attention - but still allow him to explore. Our best efforts were made, but maybe we could have narrowed it down even more in some areas and expanded the interactive aspects in others.

Contributions. Who did what?

Spyridon Vlachospyros (s213160): Responsible for formatting the final notebook and most of its text. Responsible for interactive barplot, some plots in the begining and part of the reggresion. From the part A of the assignment was responsible for recording the video and colect some initial datasets

Jonas Vestergaard (s162615) was responsible for finding appropiate additional data sets to accompany the recycling data, the initial exploratory analysis for part A of the assignment, creation of map plots, setting up the website, inserting content into the website, the plots in the website (except for the bar chart and machine learning plots) and for writing the text for the website.

Elias Bobrowski (s216056): Responsible for making and formatting website, writing explainer notebook, data cleaning and research.Responsible for NYC districts and their public recycling bins interactive plot, regression model, and presentation for video introduction.

### Appendix
